In [5]:
import pandas as pd
from urllib.parse import urlparse
import tldextract

In [6]:
df = pd.read_csv('..//data/raw/restructured_all_cves.csv')
df.shape

(269509, 25)

In [7]:
df.head()

,CVE ID,Source Identifier,Published Date,Last Modified Date,Vulnerability Status,Description,CVSS Version,CVSS Vector String,Access Vector,Access Complexity,...,Base Severity,Exploitability Score,Impact Score,acInsufInfo,Obtain All Privilege,Obtain User Privilege,Obtain Other Privilege,User Interaction Required,CWE ID,Reference URLs
0,CVE-1999-0095,cve@mitre.org,1988-10-01T04:00:00.000,2019-06-11T20:29:00.263,Modified,"The debug command in Sendmail is enabled, allo...",2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NETWORK,LOW,...,HIGH,10.0,10.0,False,True,False,False,False,NVD-CWE-Other,http://seclists.org/fulldisclosure/2019/Jun/16...
1,CVE-1999-0082,cve@mitre.org,1988-11-11T05:00:00.000,2008-09-09T12:33:40.853,Analyzed,CWD ~root command in ftpd allows root access.,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NETWORK,LOW,...,HIGH,10.0,10.0,False,True,False,False,False,NVD-CWE-Other,http://www.alw.nih.gov/Security/Docs/admin-gui...
2,CVE-1999-1471,cve@mitre.org,1989-01-01T05:00:00.000,2008-09-05T20:19:36.257,Analyzed,Buffer overflow in passwd in BSD based operati...,2.0,AV:L/AC:L/Au:N/C:C/I:C/A:C,LOCAL,LOW,...,HIGH,3.9,10.0,False,True,False,False,False,NVD-CWE-Other,http://www.cert.org/advisories/CA-1989-01.html...
3,CVE-1999-1122,cve@mitre.org,1989-07-26T04:00:00.000,2018-05-03T01:29:04.817,Modified,Vulnerability in restore in SunOS 4.0.3 and ea...,2.0,AV:L/AC:L/Au:N/C:P/I:P/A:P,LOCAL,LOW,...,MEDIUM,3.9,6.4,False,False,False,False,False,NVD-CWE-Other,http://www.cert.org/advisories/CA-1989-02.html...
4,CVE-1999-1467,cve@mitre.org,1989-10-26T04:00:00.000,2017-12-19T02:29:08.393,Modified,Vulnerability in rcp on SunOS 4.0.x allows rem...,2.0,AV:N/AC:L/Au:N/C:C/I:C/A:C,NETWORK,LOW,...,HIGH,10.0,10.0,False,True,False,False,False,NVD-CWE-Other,http://www.cert.org/advisories/CA-1989-07.html...


In [8]:
cwe_df = df['Vulnerability Status'].value_counts()
cwe_df

Vulnerability Status
Analyzed               137089
Modified                97813
Awaiting Analysis       19454
Rejected                14466
Received                  347
Undergoing Analysis       340
Name: count, dtype: int64

In [9]:
cwe_df = df['CWE ID'].value_counts()
cwe_df

CWE ID
CWE-79            30016
NVD-CWE-Other     28999
NVD-CWE-noinfo    28977
CWE-89            12849
CWE-119           11567
                  ...  
CWE-646               1
CWE-64                1
CWE-465               1
CWE-527               1
CWE-615               1
Name: count, Length: 554, dtype: int64

In [10]:
def extract_domain(url):
    """
    Extract domain from a URL.
    :param url: The URL to extract the domain from
    :return: The domain (e.g., 'github', 'google')
    """
    extracted = tldextract.extract(url)
    return f"{extracted.domain}.{extracted.suffix}"

def process_urls(urls):
    """
    Process a semicolon-separated string of URLs and extract domains.
    :param urls: Semicolon-separated string of URLs
    :return: List of domains
    """
    if pd.isna(urls) or urls.strip() == '':
        return []
    # Split the URLs by semicolon and extract domains
    urls_list = urls.split(';')
    domains = [extract_domain(url.strip()) for url in urls_list if url.strip()]
    return domains

def categorize_and_count(df_column):
    """
    Categorize URLs by their domains and count the occurrences.
    :param df_column: DataFrame column containing URLs
    :return: Series with domain counts
    """
    # Apply the processing function to the DataFrame column
    all_domains = [domain for urls in df_column for domain in process_urls(urls)]
    
    # Count occurrences of each domain
    return pd.Series(all_domains).value_counts()

# Apply the function and get the counts
domain_counts = categorize_and_count(df['Reference URLs'])

print("All urls in total:", domain_counts.sum())

domain_counts_df = domain_counts.reset_index()
domain_counts_df.columns = ['Domain', 'Count']
domain_counts_df

All urls in total: 924834


,Domain,Count
0,securityfocus.com,83636
1,github.com,66059
2,secunia.com,59764
3,redhat.com,38702
4,ibmcloud.com,37768
...,...,...
11365,barmat.io,1
11366,greysec.net,1
11367,ucr.edu,1
11368,acrolinx.com,1


### Count rows containing every domain

In [11]:
def remove_duplicate_domains(url_string):
    if pd.isna(url_string):  # Check if the value is NaN or None
        return ''  # Return an empty string or handle as needed
    urls = url_string.split(';')  # Split the URL string into a list
    seen_domains = set()
    unique_urls = []

    for url in urls:
        domain = urlparse(url).netloc
        if domain not in seen_domains:
            seen_domains.add(domain)
            unique_urls.append(url)

    return ';'.join(unique_urls)  # Join the unique URLs back into a string, separated by semicolons

# Avoid change origin data 
temp_df = df
# Apply the function to each row in the DataFrame
temp_df['Reference URLs'] = temp_df['Reference URLs'].apply(remove_duplicate_domains)

domain_row_counts = categorize_and_count(temp_df['Reference URLs'])

domain_counts_df = domain_row_counts.reset_index()

domain_counts_df.columns = ['Domain', 'Count']
domain_counts_df

,Domain,Count
0,securityfocus.com,68415
1,github.com,46055
2,ibmcloud.com,36533
3,secunia.com,31482
4,securitytracker.com,28161
...,...,...
11304,liftsecurity.io,1
11305,perspectiverisk.com,1
11306,cybelesoft.com,1
11307,mantz-it.com,1


In [12]:
domain_counts_df.to_csv('..//data/raw/all_domain_list.csv')